In [1]:
from torchdrug import transforms
from torchdrug import data, core, layers, tasks, metrics, utils, models
from torchdrug.layers import functional
from torchdrug.core import Registry as R

import torch
from torch.utils import data as torch_data
from torch.nn import functional as F

/data/project/aigenintern/aigenintern2/miniconda3/envs/jc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from lib.datasets import ATPBind

truncuate_transform = transforms.TruncateProtein(max_length=350, random=False)
protein_view_transform = transforms.ProteinView(view='residue')
transform = transforms.Compose([truncuate_transform, protein_view_transform])

dataset = ATPBind(atom_feature=None, bond_feature=None, residue_feature="default", transform=transform)

train_set, valid_set, test_set = dataset.split()
print("train samples: %d, valid samples: %d, test samples: %d" % (len(train_set), len(valid_set), len(test_set)))

Split num:  [346, 42, 41]
train samples: 346, valid samples: 42, test samples: 41


In [5]:
from lib.tasks import NodePropertyPrediction

def main():
    model = models.ProteinCNN(input_dim=21,
                        hidden_dims=[1024, 1024],
                        kernel_size=5, padding=2, readout="max")

    task = NodePropertyPrediction(model, normalization=False, num_mlp_layer=2, metric=("micro_auroc", "micro_auprc", "macro_auprc", "macro_auroc"))
    optimizer = torch.optim.Adam(task.parameters(), lr=1e-4)
    solver = core.Engine(task, train_set, valid_set, test_set, optimizer, batch_size=4, log_interval=10000000, gpus=[2])
    solver.train(num_epoch=100)
    solver.evaluate("valid")
    solver.evaluate("test")

In [6]:
main()

14:20:10   Preprocess training set
14:20:15   {'batch_size': 4,
 'class': 'core.Engine',
 'gpus': [2],
 'gradient_interval': 1,
 'log_interval': 10000000,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.0001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'class': 'NodePropertyPrediction',
          'criterion': 'bce',
          'metric': ('micro_auroc',
                     'micro_auprc',
                     'macro_auprc',
                     'macro_auroc'),
          'model': {'activation': 'relu',
                    'class': 'models.ProteinConvolutionalNetwork',
                    'concat_hidden': False,
                    'hidden_dims': [1024, 1024],
                    'input_dim': 21,
                    'kernel_size': 5,
                    'padding': 2,
                    'readout': 'max',
              